In [ ]:
!pip install sentence-transformers
!pip install pdfminer.six
!pip install unstructured
!pip install transformers

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from sentence_transformers import SentenceTransformer
import requests
from urllib.parse import urlparse
from pathlib import Path

In [ ]:
def create_chunks(text, chunk_size=1024):
    tokens = word_tokenize(text)
    chunks = [' '.join(tokens[i:i+chunk_size]) for i in range(0, len(tokens), chunk_size)]
    return chunks

In [ ]:
model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')

def vectorize_chunks(chunks):
    return model.encode(chunks)                        

In [ ]:
from pdfminer.high_level import extract_text

def convert_pdf_to_text(pdf_path):
    text = extract_text(pdf_path)
    return text

pdf_path = 'data/PepsiCo 10k.pdf'
document_text = convert_pdf_to_text(pdf_path)

In [ ]:
chunks = create_chunks(document_text, chunk_size=1024)

chunk_vectors = vectorize_chunks(chunks)

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('data/financebench_sample_150.csv')

# Display the first few rows to verify 
print(df.head())

In [ ]:
# Filter queries for the specific document
pepsico_queries = df[df['doc_name'] == 'PEPSICO_2022_10K']

# Extract the queries and convert them to a list
query_texts = pepsico_queries['question'].tolist()

In [ ]:
query_vectors = model.encode(query_texts)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_top_k_chunks_for_all_queries(query_vectors, chunk_vectors, k=5):
    all_top_chunks_indices = []
    for query_vector in query_vectors:
        similarities = cosine_similarity([query_vector], chunk_vectors)[0]
        top_k_indices = similarities.argsort()[-k:][::-1]
        all_top_chunks_indices.append(top_k_indices)
    return all_top_chunks_indices

# Retrieve top-k chunks for all queries
all_top_chunks_indices = retrieve_top_k_chunks_for_all_queries(query_vectors, chunk_vectors, k=5)

In [ ]:
def display_top_chunks_for_queries(queries, all_top_chunks_indices, chunks):
    for i, query in enumerate(queries):
        print(f"Query: {query}")
        print("Top Chunks:")
        top_indices = all_top_chunks_indices[i]
        for idx in top_indices:
            print(f"- Chunk {idx}:\n{chunks[idx]}")
        print("\n" + "-"*50 + "\n")

# Assuming 'query_texts' contains the text of your queries
display_top_chunks_for_queries(query_texts, all_top_chunks_indices, chunks)

In [ ]:
import matplotlib.pyplot as plt

# Data setup
scenarios = ['Fixed-size Chunk & Vectorize', 'Direct Access to Evidence']
accuracies = [0, 80]

# Creating the plot
plt.figure(figsize=(10, 6))
plt.bar(scenarios, accuracies, color=['red', 'blue', 'green'])

# Adding titles and labels
plt.title('Comparison of Retrieval Techniques')
plt.xlabel('Scenario')
plt.ylabel('Accuracy (%)')
plt.ylim(0, 100)

# Adding text labels above the bars
for i, accuracy in enumerate(accuracies):
    plt.text(i, accuracy + 3, f'{accuracy}%', ha='center', va='bottom')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Define the destination folder
dest_folder = './data'
Path(dest_folder).mkdir(exist_ok=True) # Ensure the destination folder exists

def download_document(url, dest_folder, doc_name):
    # Parse the URL to get a meaningful filename
    filename = f"{doc_name}.pdf" if not doc_name.lower().endswith('.pdf') else doc_name

    # Construct the full path where the file will be saved
    filepath = Path(dest_folder) / filename

    # Check if file already exists to avoid re-downloading
    if not filepath.exists():
        try:
            # Make HTTP GET request to download the document
            response = requests.get(url)
            response.raise_for_status() # Ensure the request was succesful

            # Write the document to a file
            with open(filepath, 'wb') as file:
                file.write(response.content)
            print(f"Downloaded {filename} to {dest_folder}")
        except requests.exceptions.RequestException as e: # Catch any request related errors
            print(f"Failed to download {url}. Error: {e}")
    else:
        print(f"{filename} already exists in {dest_folder}")

# Load the dataset
df = pd.read_csv('data/financebench_sample_150.csv')

# Download documents
for _, row in df.iterrows():
    if pd.notna(row['doc_link']) and pd.notna(row['doc_name']):
        download_document(row['doc_link'], dest_folder, row['doc_name'])

In [ ]:
# Create a Path object
dest_path = Path(dest_folder)

# List all PDF files in the destination folder
downloaded_files = [file.name for file in dest_path.iterdir() if file.suffix == '.pdf']

# Print the number of downloaded files
print(f"Number of downloaded files: {len(downloaded_files)}")

In [ ]:
import os

pdf_folder_path = 'data/reports'
extracted_texts_path = os.path.join(pdf_folder_path, "extracted_texts")
document_texts = {}

# Ensure the directory for extracted texts exists
os.makedirs(extracted_texts_path, exist_ok=True)

for doc_name in df['doc_name'].unique():
    pdf_path = os.path.join(pdf_folder_path, f"{doc_name}.pdf")
    output_text_path = os.path.join(extracted_texts_path, f"{doc_name}.txt")

    # Check if the PDF file exists before proceeding
    if os.path.exists(pdf_path):
    
        # Check if the text has already been extracted and saved
        if not os.path.exists(output_text_path):  # If not, extract and save
            print(f"Extracting text from {pdf_path}")
            document_text = convert_pdf_to_text(pdf_path)
            with open(output_text_path, 'w', encoding='utf-8') as text_file:
                text_file.write(document_text)
            print(f"Text saved to {output_text_path}")
        else:
            print(f"Text already extracted for {doc_name}")
    
        # Load the extracted text into memory
        with open(output_text_path, 'r', encoding='utf-8') as text_file:
            document_texts[doc_name] = text_file.read()

    else:
        print(f"PDF file does not exist: {pdf_path}")

In [ ]:
extracted_files_count = len([name for name in os.listdir(extracted_texts_path) if os.path.isfile(os.path.join(extracted_texts_path, name))])
print(f"Number of extracted text files: {extracted_files_count}")

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['doc_name_encoded'] = encoder.fit_transform(df['doc_name'])

In [ ]:
from sklearn.model_selection import train_test_split

X = df['question'] # Features
y = df['doc_name_encoded'] # Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
import random

# Sample 5 random rows from the dataframe
sample_questions = df.sample(n=5)

# Print the selected questions and their doc_name
for index, row in sample_questions.iterrows():
    print(f"Question: {row['question']}")
    print(f"Document Name: {row['doc_name']}\n")

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

In [ ]:
encoder = LabelEncoder()
df['doc_name_encoded'] = encoder.fit_transform(df['doc_name'])

In [ ]:
import spacy
from sklearn.pipeline import Pipeline

# Load Spacy's NLP model
nlp = spacy.load("en_core_web_sm")

def extract_keywords_with_ner(text):
    # Process the text with Spacy NLP pipeline
    doc = nlp(text)

    # Extract entities identified by Spacy NER
    keywords = [ent.text for ent in doc.ents if ent.label_ in ["ORG"]]

    # Join keywords for TF-IDF vectorization
    keywords_str = ' '.join(keywords)
    return keywords_str

# Apply keyword extraction on questions
df['question_keywords'] = df['question'].apply(extract_keywords_with_ner)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['question_keywords'], df['doc_name_encoded'], test_size=0.2, random_state=42)

# Creating a TF-IDF Vectorizer to convert the questions into vectors
tfidf_vectorizer = TfidfVectorizer()

# Creating a Logistic Regression model
logistic_regression_model = LogisticRegression(max_iter=1000)

# Creating a pipeline that first converts texts to vectors and then applies logistic regression
pipeline = Pipeline([
    ('tfidf', tfidf_vectorizer),
    ('logistic_regression', logistic_regression_model)
])

# Training the pipeline
pipeline.fit(X_train, y_train)

# Testing the model
accuracy = pipeline.score(X_test, y_test)
print(f'Model Accuracy: {accuracy}')

# Predicting document names
predicted_doc_names_encoded = pipeline.predict(X_test)
predicted_doc_names = encoder.inverse_transform(predicted_doc_names_encoded)

# Example of displaying a few predictions
for i, (question, prediction) in enumerate(zip(X_test[:5], predicted_doc_names[:5])):
    print(f"Question: {question}\nPredicted Document Name: {prediction}\n{'-'*60}")


In [ ]:
# Data setup
scenarios = ['Baseline Model', 'NER Enhanced Model']
accuracies = [13.333333, 13.333333]

# Creating the plot
plt.figure(figsize=(8, 5))
plt.bar(scenarios, accuracies, color=['skyblue', 'red'])

# Adding titles and labels
plt.title('Comparison of Document Retrieval Models')
plt.xlabel('Model')
plt.ylabel('Accuracy (%)')
plt.ylim(0, 100)

# Adding text labels above the bars
for i, accuracy in enumerate(accuracies):
    plt.text(i, accuracy + 0.5, f'{accuracy: .2f}%', ha='center', va='bottom')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
!pip install openai

In [ ]:
import openai

# Set up API key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
client = openai.OpenAI(api_key=openai.api_key)

In [ ]:
def format_doc_name(data):
    try:
        json_data = eval(data)
        return f"{json_data['company']}_{json_data['year']}_{json_data['financial_report_type'].replace('-', '')}.pdf"
    except Exception as e:
        print(f"Error formatting document name: {e}")
        return None

In [ ]:
X = df['question']
y = df['doc_name']

In [ ]:
import json

def find_financial_report(question):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": "You are a financial analyst. Return information in JSON format with the relevant fields of company, year, and financial report type."},
                {"role": "user", "content": f"Which standard financial report (e.g., 10-K, 10-Q, 8-K) typically contains the information needed to answer this question: '{question}'?"}
            ]
        )
        # Extract and return the JSON formatted text from the response
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def format_doc_name(data):
    try:
        json_data = json.loads(data)
        
        # Normalize the company name
        company = json_data.get('company', 'Unknown').replace(' ', '').replace('(', '').replace(')', '')

        # Handle the year and report type formatting
        year = json_data.get('year', 'Unknown')
        if isinstance(year, list):
            year = max(year, key=lambda x: int(x.replace('FY', '').replace('Q', '').split(' ')[0]))  # Get the latest year
        year = str(year).replace('FY', '')  # Clean up any fiscal year notation

        # Handle quarterly data
        if 'Q' in year:
            parts = year.split('Q')
            year = parts[0] + 'Q' + parts[1]

        financial_report_type = json_data.get('financial_report_type', '').replace('-', '')
        
        # Normalize the year to four digits
        if len(year) == 2:  # Handle cases like '22' for 2022
            year = '20' + year

        return f"{company}_{year}_{financial_report_type}"
    except Exception as e:
        print(f"Error in format_doc_name: {e}")
        return None

def get_predictions(questions):
    predictions = []
    for question in questions:
        print(f"Processing question: {question}")
        report_type = find_financial_report(question)
        if report_type:
            print(f"Report type found: {report_type}")
            doc_name = format_doc_name(report_type)
            if doc_name:
                predictions.append(doc_name)
                print(f"Formatted document name: {doc_name}")  # Print the document name here
            else:
                predictions.append('Formatting error')
                print("Formatting error")  # Indicate a formatting error
        else:
            predictions.append('Unknown')
            print("Unknown report type")  # Indicate that no report type was found
    return predictions

In [ ]:
# Sample 5 random rows from the dataframe
sample_questions = df.sample(n=30)

# Print the selected questions and their doc_name
for index, row in sample_questions.iterrows():
    print(f"Question: {row['question']}")
    print(f"Document Name: {row['doc_name']}\n")

# Extract only the question text to send to the prediction function
question_texts = sample_questions['question'].tolist()
actual_names = sample_questions['doc_name'].tolist()

# Debug print to check what questions are being sent to the prediction function
print("Questions being processed:", question_texts)

y_pred_sample = get_predictions(question_texts)
print("Sample Predictions:", y_pred_sample)

In [ ]:
# Actual and Predicted document names
actual_names = [
    "AMERICANEXPRESS_2022_10K", "MGMRESORTS_2023Q2_10Q", "JOHNSON&JOHNSON_2022Q4_EARNINGS",
    "COSTCO_2021_10K", "LOCKHEEDMARTIN_2021_10K", "3M_2023Q2_10Q",
    "CORNING_2020_10K", "Pfizer_2023Q2_10Q", "ADOBE_2022_10K",
    "LOCKHEEDMARTIN_2022_10K", "PEPSICO_2023_8K_dated-2023-05-05", "ADOBE_2022_10K",
    "AMD_2022_10K", "WALMART_2018_10K", "GENERALMILLS_2020_10K",
    "PFIZER_2021_10K", "PEPSICO_2021_10K", "BOEING_2018_10K",
    "VERIZON_2022_10K", "AMD_2022_10K", "3M_2023Q2_10Q",
    "AMCOR_2023Q2_10Q", "VERIZON_2022_10K", "JPMORGAN_2021Q1_10Q",
    "NIKE_2023_10K", "AES_2022_10K", "NIKE_2019_10K",
    "PEPSICO_2023Q1_EARNINGS", "AMERICANEXPRESS_2022_10K", "BESTBUY_2023_10K"
]

predicted_names = [
    "AmericanExpress_2022_10K", "MGM_H1 2023_10Q", "Johnson&JohnsonJnJ_2022_10K", 
    "Costco_2021_10K", "LockheedMartin_2021_10K", "3M_2023_10Q",
    "Corning_Unknown_", "Pfizer_current or upcoming fiscal year_10K", "Adobe_2022_10K",
    "LockheedMartin_2020-2022_10K", "Pepsico_2023_proxy statement (DEF 14A)", "Adobe_2022_10K",
    "AMD_2022_10K", "Walmart_2018_10K", "GeneralMills_2020_",
    "Pfizer_2019_10K", "PepsiCo_2021_10K", "Boeing_2018_10K",
    "Verizon_2022_10K", "AMD_2022_10K", "3M_2023_10Q",
    "AMCOR_2023_10Q", "Verizon_2022_10K", "JPMorganChase&Co.JPM_2021_10Q",
    "Nike_2023_10K", "AESCorporation_2022_10K", "Nike_2019_10K",
    "Pepsico_2023_8K", "AmericanExpress_2022_10K", "BestBuy_Unknown_"
]

# Normalizing both lists to ensure case and spacing are not an issue
actual_names_normalized = [name.replace(" ", "").upper() for name in actual_names]
predicted_names_normalized = [name.replace(" ", "").upper() for name in predicted_names]

# Calculating accuracy
correct_predictions = sum(1 for actual, predicted in zip(actual_names_normalized, predicted_names_normalized) if actual == predicted)
total_predictions = len(actual_names)
accuracy_percentage = (correct_predictions / total_predictions) * 100

print(f"Correct Predictions: {correct_predictions}")
print(f"Total Predictions: {total_predictions}")
print(f"Accuracy: {accuracy_percentage:.2f}%")

In [ ]:
import re

def find_relevant_document(question, doc_names):
    try:
        doc_list = ', '.join(doc_names)  # List of all document names

        prompt = f"Given these financial documents: {doc_list}, which should be used to answer the question: '{question}'?"
        print("Prompt sent to model:", prompt)

        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are a financial analyst. Decide which financial document to use."},
                {"role": "user", "content": prompt}
            ]
        )

        predicted_report = response.choices[0].message.content.strip()
        print("Predicted Report:", predicted_report)

        # Dynamic regex pattern based on document names
        doc_names_regex = '|'.join(re.escape(doc) for doc in doc_names)
        pattern = rf'({doc_names_regex})'
        match = re.search(pattern, predicted_report)
        matched_document = match.group(1) if match else None

        if matched_document:
            print("Matched Document:", matched_document)
        else:
            print("No matching document found.")

        return matched_document

    except Exception as e:
        print(f"An error occurred while finding the relevant document: {e}")
        return None

def test_document_matching(df, num_samples=30):
    samples = df.sample(n=num_samples)
    
    # Collect all document names from the DataFrame to provide as options
    all_doc_names = df['doc_name'].unique().tolist()

    for index, sample in samples.iterrows():
        question = sample['question']
        actual_doc_name = sample['doc_name'].strip()

        predicted_doc_name = find_relevant_document(question, all_doc_names)  # Pass all possible documents

        is_correct = predicted_doc_name == actual_doc_name
        status = "Correct" if is_correct else "Incorrect"

        print(f"Question: {question}")
        print(f"Predicted Document: {predicted_doc_name}")
        print(f"Actual Document: {actual_doc_name}")
        print(f"Status: {status}")

# Example usage
num_questions_to_test = 30  # Adjust the number of tests as needed
test_document_matching(df, num_questions_to_test)

In [ ]:
Accuracy: 24/30

In [ ]:
def find_relevant_document(question, doc_names):
    try:
        doc_list = ', '.join(doc_names)  # List of all document names

        prompt = f"Given these financial documents: {doc_list}, which should be used to answer the question: '{question}'?"
        print("Prompt sent to model:", prompt)

        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are a financial analyst. Decide which financial document to use."},
                {"role": "user", "content": prompt}
            ]
        )

        predicted_report = response.choices[0].message.content.strip()
        print("Predicted Report:", predicted_report)  # Debugging output

        # Dynamic regex pattern to extract document names and their years/quarters
        doc_names_regex = '|'.join(re.escape(doc) for doc in doc_names)
        pattern = rf'({doc_names_regex})'
        matches = re.findall(pattern, predicted_report)
        
        if matches:
            # Parse years and sort documents by recency
            def extract_year(doc):
                match = re.search(r'(\d{4})', doc)
                return int(match.group(1)) if match else 0

            sorted_docs = sorted(matches, key=extract_year, reverse=True)
            matched_document = sorted_docs[0] if sorted_docs else None
            print("Matched Document:", matched_document)
        else:
            matched_document = None
            print("No matching document found.")

        return matched_document

    except Exception as e:
        print(f"An error occurred while finding the relevant document: {e}")
        return None

def test_document_matching(df, num_samples=30):
    samples = df.sample(n=num_samples)
    all_doc_names = df['doc_name'].unique().tolist()

    for index, sample in samples.iterrows():
        question = sample['question']
        predicted_doc_name = find_relevant_document(question, all_doc_names)
        actual_doc_name = sample['doc_name'].strip()

        is_correct = predicted_doc_name == actual_doc_name
        status = "Correct" if is_correct else "Incorrect"

        print(f"Question: {question}")
        print(f"Predicted Document: {predicted_doc_name}")
        print(f"Actual Document: {actual_doc_name}")
        print(f"Status: {status}")

# Example usage
num_questions_to_test = 30  # Adjust the number of tests as needed
test_document_matching(df, num_questions_to_test)

In [ ]:
Accuracy: 25/30